<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%208%20-%20Reinforcement%20Learning%20para%20problemas%20de%20Stock%20Market%20Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno

In [2]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

     |████████████████████████████████| 332.1MB 60kB/s 
     |████████████████████████████████| 3.0MB 30.1MB/s 
     |████████████████████████████████| 419kB 39.4MB/s 


In [0]:
!pip install pandas-datareader

## Paso 2: Importar las dependencias del proyecto

In [0]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [4]:
tf.__version__

'2.0.0-alpha0'

## Paso 3: Construir la red neuronal de la AI del Trader 

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Manten, Compra, Vende
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Paso 4: Pre procesado del dataset

### Definir las funciones adicionales

#### Sigmoide

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Función de formato de precios

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Carga del dataset

In [0]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

### State creator

In [0]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = list(data[starting_id:timestep+1])
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

### Cargar una divisa de mercado

In [0]:
#stock_name = "AAPL"
stock_name = "BTC-USD"
data = dataset_loader(stock_name)

In [0]:
data.head()

Date
2010-01-04    30.572857
2010-01-05    30.625713
2010-01-06    30.138571
2010-01-07    30.082857
2010-01-08    30.282858
Name: Close, dtype: float64

## Paso 5: Entrenar la AI Trader

### Configurar los hyper parámetros

In [0]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

### Definir el modelo del AI Trader

In [0]:
trader = AI_Trader(window_size)

W0801 19:19:23.745935 139627749189504 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Bucle de entrenamiento

In [0]:
for episode in range(1, episodes + 1):
  
  print("Episodio: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Compra
      trader.inventory.append(data[t])
      print("AI Trader compró: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Vende
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader vendió: ", stocks_price_format(data[t]), " Beneficio: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("BENEFICIO TOTAL: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/2410 [00:00<?, ?it/s]

Episodio: 1/1000
AI Trader compró:  $ 30.625713
AI Trader compró:  $ 30.138571
AI Trader compró:  $ 30.282858
AI Trader compró:  $ 30.015715
AI Trader vendió:  $ 30.092857  Beneficio: - $ 0.532856
AI Trader vendió:  $ 29.724285  Beneficio: - $ 0.414286
AI Trader vendió:  $ 28.250000  Beneficio: - $ 2.032858
AI Trader compró:  $ 29.010000
AI Trader compró:  $ 29.420000
AI Trader compró:  $ 29.697144
AI Trader compró:  $ 28.469999
AI Trader compró:  $ 27.437143
AI Trader vendió:  $ 27.980000  Beneficio: - $ 2.035715
AI Trader compró:  $ 28.461428
AI Trader vendió:  $ 27.435715  Beneficio: - $ 1.574286
AI Trader vendió:  $ 27.731428  Beneficio: - $ 1.688572
AI Trader compró:  $ 28.027143
AI Trader vendió:  $ 28.381428  Beneficio: - $ 1.315716
AI Trader compró:  $ 28.625713
AI Trader vendió:  $ 29.057142  Beneficio: $ 0.587143
AI Trader compró:  $ 28.935715


  1%|▏         | 34/2410 [00:04<05:57,  6.64it/s]

AI Trader compró:  $ 28.631428
AI Trader vendió:  $ 28.151428  Beneficio: $ 0.714285


  2%|▏         | 37/2410 [00:05<06:41,  5.91it/s]

...

AI Trader vendió:  $ 65.712860  Beneficio: $ 0.307144
AI Trader vendió:  $ 64.121429  Beneficio: - $ 1.214287


 33%|███▎      | 789/2410 [02:17<04:45,  5.67it/s]

AI Trader vendió:  $ 63.722858  Beneficio: - $ 4.838573
AI Trader vendió:  $ 64.401428  Beneficio: - $ 2.254288


 63%|██████▎   | 1511/2410 [04:23<02:35,  5.78it/s]

Buffered data was truncated after reaching the output size limit.